In [74]:
import requests
from bs4 import BeautifulSoup
import csv
import xml.etree.ElementTree as ET
import time

from requests.auth import HTTPBasicAuth

In [93]:
consumer_key = 'Lq3V1hAxII2Fv8YSdcrhfA538HzhsfzhelYqSUgzGU0A8AEU'
consumer_secret_key = 'jHJjRSzVQBjCizDYmFg4cqWza2R8TOCcLaecRbXrJQcseLu8Lr0AqYXL1ldkAx8A'

# EPO OAuth2 token URL
token_url = "https://ops.epo.org/3.2/auth/accesstoken"

# Request access token
response = requests.post(
    token_url,
    auth=HTTPBasicAuth(consumer_key, consumer_secret_key),
    data={"grant_type": "client_credentials"},
)

# Extract token
if response.status_code == 200:
    token = response.json()["access_token"]
    print("Access Token:", token)
else:
    print("Failed to get token:", response.text)

Access Token: wAXva3VJLXpMSpjFvtyK3WkBCRa3


In [76]:
def search_for_patents_by_year(year, token, start, end):
    
    search_url = "https://ops.epo.org/rest-services/published-data/search"

    # Headers with authentication
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml",
    }
    
    # Parameters for the search

    params = {
        "q": f'pd within "{year},{year}"',
        "Range": f"{start}-{end}"  
    }
    
    response = requests.get(search_url, headers=headers, params=params)
    
    if response.status_code == 200:
        root = ET.fromstring(response.content)

        # Namespace dictionary (important for parsing)
        ns = {"epo": "http://www.epo.org/exchange"}

        # Extract all document-id elements
        patents = []
        for doc in root.findall(".//epo:document-id", namespaces=ns):
            country = doc.find("epo:country", namespaces=ns)
            doc_number = doc.find("epo:doc-number", namespaces=ns)

            if country is not None and doc_number is not None:
                patents.append({
                "country": country.text,
                "doc_number": doc_number.text
            })

        return patents  # Returns a dictionary {doc-number: country}

    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

        
#print(len(search_for_patents_by_year(1883, token, 2001, 2002)))


In [95]:
all_patents = []

for year in range(1977, 2026):
    start = 1
    end = 100
    expected_length = 100
    
    patents = []
    
    while True:
        try:
            batch = search_for_patents_by_year(year, token, start, end)
            
            if len(batch) < expected_length:
                patents.extend(batch)
                break

            patents.extend(batch)
            start += 100
            end += 100
            time.sleep(1.5)
            #print(len(patents))

        except Exception as e:
            break

    all_patents.append(patents) 
    time.sleep(1.5)
    print(year)               

    




Error: 403, <?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<fault xmlns="http://ops.epo.org">
    <code>CLIENT.RobotDetected</code>
    <message>Recent behaviour implies you are a robot. The server is at the moment too busy to serve robots. Please try again later</message>
</fault>

1977
Error: 403, <?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<fault xmlns="http://ops.epo.org">
    <code>CLIENT.RobotDetected</code>
    <message>Recent behaviour implies you are a robot. The server is at the moment too busy to serve robots. Please try again later</message>
</fault>



KeyboardInterrupt: 

In [84]:
print(len(all_patents))

134


In [ ]:
csv_filename = "patents_2.csv"

with open(csv_filename, mode="a", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    
    # Write header
    writer.writerow(["year", "country", "doc_number"])
    
    # Flatten the data while writing
    for i in range (len(all_patents)):
        year = 1977 + i
        patent_list = all_patents[i]

        for patent in patent_list:
            writer.writerow([year, patent["country"], patent["doc_number"]])

print(f"CSV file '{csv_filename}' has been created successfully.")

CSV file 'patents.csv' has been created successfully.
